In [1]:
import pandas as pd
import spacy
import networkx as nx
import os
from tqdm import tqdm
from joblib import Parallel, delayed

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
def extract_genome_info(text):
    doc = nlp(text)
    genome_info = []
    for ent in doc.ents:
        if ent.label_ == 'GENE_OR_GENE_PRODUCT':
            genome_info.append(ent.text)
    return genome_info

In [3]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger'])

In [4]:
metadata_path = '/kaggle/input/CORD-19-research-challenge/metadata.csv'

In [5]:
relevant_articles = []
chunk_size = 1000
for chunk in tqdm(pd.read_csv(metadata_path, chunksize=chunk_size), total=len(pd.read_csv(metadata_path))):
    filtered_chunk = chunk[chunk['abstract'].str.contains('SARS-CoV-2', na=False, case=False) | 
                           chunk['title'].str.contains('SARS-CoV-2', na=False, case=False)]
    relevant_articles.append(filtered_chunk)

/tmp/ipykernel_20/874142979.py:3: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in tqdm(pd.read_csv(metadata_path, chunksize=chunk_size), total=len(pd.read_csv(metadata_path))):
  0%|          | 1057/1056660 [01:36<26:42:50, 10.98it/s]


In [6]:
relevant_articles_df = pd.concat(relevant_articles)

In [7]:
def process_chunk(chunk):
    genome_info = []
    for idx, row in chunk.iterrows():
        try:
            abstract = row['abstract']
            if isinstance(abstract, str):
                doc = nlp(abstract)  # Process the abstract with spaCy NLP
                for ent in doc.ents:
                    if ent.label_ == 'GENE_OR_GENE_PRODUCT' and 'SARS-CoV-2' in ent.text:
                        genome_info.append((row['cord_uid'], ent.text))
        except Exception as e:
            print(f"Error processing row with cord_uid {row['cord_uid']}: {e}")
    return genome_info

In [8]:
chunk_size = 100

In [9]:
metadata_path = '/kaggle/input/CORD-19-research-challenge/metadata.csv'
metadata_df = pd.read_csv(metadata_path)

/tmp/ipykernel_20/2531542173.py:2: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(metadata_path)


In [10]:
relevant_articles_df = metadata_df[metadata_df['abstract'].str.contains('SARS-CoV-2', case=False, na=False)]

# Limit the number of articles for testing (optional)
relevant_articles_df = relevant_articles_df.head(1000)

In [11]:
chunks = [relevant_articles_df[i:i + chunk_size] for i in range(0, len(relevant_articles_df), chunk_size)]

In [12]:
num_cores = os.cpu_count()
processed_chunks = Parallel(n_jobs=num_cores)(delayed(process_chunk)(chunk) for chunk in tqdm(chunks, total=len(chunks)))

 40%|████      | 4/10 [00:00<00:00,  9.59it/s]/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy ve

Error processing row with cord_uid 72s8wpla: buffer source array is read-only
Error processing row with cord_uid ehw61gu6: buffer source array is read-only
Error processing row with cord_uid puc13jf1: buffer source array is read-only
Error processing row with cord_uid ya9ivlry: buffer source array is read-only
Error processing row with cord_uid 4wuv6ntb: buffer source array is read-only
Error processing row with cord_uid unzlkly4: buffer source array is read-only
Error processing row with cord_uid zyghu7wk: buffer source array is read-only
Error processing row with cord_uid 4i9us8ox: buffer source array is read-only
Error processing row with cord_uid x49e0312: buffer source array is read-only
Error processing row with cord_uid 7jpw2o6r: buffer source array is read-only
Error processing row with cord_uid 1umbwskb: buffer source array is read-only
Error processing row with cord_uid nbuktgzm: buffer source array is read-only
Error processing row with cord_uid 9lu8c8o9: buffer source array

In [13]:
genome_info_all = [item for sublist in processed_chunks for item in sublist]

# Create a DataFrame to store the genome information
genome_df = pd.DataFrame(genome_info_all, columns=['cord_uid', 'genome_info'])

# Save the results to a CSV file
genome_df.to_csv('sars_cov_2_genome_info.csv', index=False)

In [14]:
processed_df = pd.concat(processed_chunks)

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [ ]:
knowledge_graph = nx.Graph()

In [ ]:
for index, row in processed_df.iterrows():
    article_id = row['cord_uid']
    genome_info = row['genome_info']
    
    knowledge_graph.add_node(article_id, title=row['title'], abstract=row['abstract'])
    for gene in genome_info:
        knowledge_graph.add_node(gene, type='GENE_OR_GENE_PRODUCT')
        knowledge_graph.add_edge(article_id, gene, relation='MENTIONS')

In [ ]:
spike_gene_articles = [node for node, data in knowledge_graph.nodes(data=True) if 'spike' in data.get('abstract', '').lower()]
print("Articles mentioning the 'spike' gene:")
for article_id in spike_gene_articles:
    print(f"- {knowledge_graph.nodes[article_id]['title']}")

In [ ]:
article_id_to_explore = 'your_article_id_here'
mentioned_genes = [neighbor for neighbor in knowledge_graph.neighbors(article_id_to_explore)
                   if knowledge_graph.nodes[neighbor].get('type') == 'GENE_OR_GENE_PRODUCT']
print(f"Genes mentioned in the article with ID '{article_id_to_explore}':")
for gene in mentioned_genes:
    print(f"- {gene}")